<a href="https://colab.research.google.com/github/AndromedaOMA/SemEval-2026_Task4-Narrative-Similarity/blob/main/SBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Accuracy check

In [ ]:
import json
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

def lin(events):
    return " [EVENT] ".join(events)

correct = 0
total = 0

with open("events_dev.jsonl") as f:
    for line in f:
        ex = json.loads(line)

        anchor = lin(ex["anchor_events"])
        A = lin(ex["A_events"])
        B = lin(ex["B_events"])

        emb = model.encode([anchor, A, B],
                            convert_to_tensor=True,
                            normalize_embeddings=True)

        scoreA = util.cos_sim(emb[0], emb[1]).item()
        scoreB = util.cos_sim(emb[0], emb[2]).item()
        ### print(f"SCORES: A:{scoreA}")
        ### print(f"SCORES: B:{scoreB}")
        ### print("="*100)
        pred_A_closer = scoreA > scoreB

        if pred_A_closer == ex["label"]:
            correct += 1
        total += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.4f} ({correct}/{total})")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Accuracy: 0.5800 (116/200)


###JSON for competition

In [ ]:
import json
from sentence_transformers import SentenceTransformer, util

INPUT_EVENTS = "events_test.jsonl"
OUTPUT_TRACK_A = "track_a.jsonl"
MODEL_NAME = "all-MiniLM-L6-v2"

model = SentenceTransformer(MODEL_NAME)

def lin(events):
    events = events or []
    return " [EVENT] ".join(events)

correct = 0
total = 0
has_label = False

with open(INPUT_EVENTS, "r", encoding="utf-8") as fin, \
     open(OUTPUT_TRACK_A, "w", encoding="utf-8") as fout:

    for line_no, line in enumerate(fin, 1):
        line = line.strip()
        if not line:
            continue

        ex = json.loads(line)

        anchor = lin(ex.get("anchor_events", []))
        A = lin(ex.get("A_events", []))
        B = lin(ex.get("B_events", []))

        emb = model.encode([anchor, A, B],
                           convert_to_tensor=True,
                           normalize_embeddings=True)

        scoreA = util.cos_sim(emb[0], emb[1]).item()
        scoreB = util.cos_sim(emb[0], emb[2]).item()

        pred_A_closer = bool(scoreA > scoreB)

        fout.write(json.dumps({"text_a_is_closer": pred_A_closer}) + "\n")

        if "label" in ex:
            has_label = True
            if pred_A_closer == bool(ex["label"]):
                correct += 1
            total += 1

print(f"[OK] Wrote: {OUTPUT_TRACK_A}")

if has_label and total > 0:
    acc = correct / total
    print(f"Dev Accuracy: {acc:.4f} ({correct}/{total})")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[OK] Wrote: track_a.jsonl


###JSON for merge

In [ ]:
import json
from sentence_transformers import SentenceTransformer, util

INPUT_EVENTS = "events_dev.jsonl"
OUTPUT_SCORES = "track_a_scores_dev.jsonl"
MODEL_NAME = "all-MiniLM-L6-v2"

model = SentenceTransformer(MODEL_NAME)

def lin(events):
    events = events or []
    return " [EVENT] ".join(events)

correct = 0
total = 0
has_label = False

with open(INPUT_EVENTS, "r", encoding="utf-8") as fin, \
     open(OUTPUT_SCORES, "w", encoding="utf-8") as fout:

    for line_no, line in enumerate(fin, 1):
        line = line.strip()
        if not line:
            continue

        ex = json.loads(line)

        anchor = lin(ex.get("anchor_events", []))
        A = lin(ex.get("A_events", []))
        B = lin(ex.get("B_events", []))

        emb = model.encode([anchor, A, B],
                           convert_to_tensor=True,
                           normalize_embeddings=True)

        scoreA = util.cos_sim(emb[0], emb[1]).item()
        scoreB = util.cos_sim(emb[0], emb[2]).item()

        pred_A_closer = bool(scoreA > scoreB)

        out = {
            "scoreA": scoreA,
            "scoreB": scoreB,
        }

        fout.write(json.dumps(out, ensure_ascii=False) + "\n")

print(f"[OK] Wrote scores to: {OUTPUT_SCORES}")


[OK] Wrote scores to: track_a_scores_dev.jsonl
